# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats


def city_mapping(team, cities):
    team = team.split(" ")
    mask = cities["NHL"].str.contains(team[-1])
    matching_rows = cities[mask]
    result_city = matching_rows["Metropolitan area"].tolist()[0]
    return result_city


def nhl_correlation():
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    cities.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    cities["NHL"] = cities["NHL"].str.replace(r"\[.*?\]", "", regex=True)
    cities = cities[["Metropolitan area", "Population", "NHL"]]
    cities = cities.sort_values(by="Metropolitan area")

    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df["team"] = nhl_df["team"].str.replace(r"\*.*", "", regex=True)
    nhl_df = nhl_df[["team", "W", "L"]]
    nhl_df = nhl_df[nhl_df["W"].str.isnumeric()]
    nhl_df["W/L"] = nhl_df["W"].astype(int) / (
        nhl_df["W"].astype(int) + nhl_df["L"].astype(int)
    )
    nhl_df["Metropolitan area"] = nhl_df["team"].apply(city_mapping, cities=cities)
    nhl_df = nhl_df.sort_values(by="Metropolitan area")
    nhl_df["W"] = pd.to_numeric(nhl_df["W"], errors="coerce")
    nhl_df["L"] = pd.to_numeric(nhl_df["L"], errors="coerce")

    cities["Population"] = pd.to_numeric(cities["Population"], errors="coerce")
    cities = pd.merge(cities, nhl_df, how="inner", on="Metropolitan area")

    # print('length of cities', len(cities))
    # print(cities)
    # print('length of nhl_df', len(nhl_df))
    # print(nhl_df)

    population_by_region = cities.groupby("Metropolitan area").agg(
        {"Population": "first"}
    )[
        "Population"
    ]  # pass in metropolitan area population from cities
    win_loss_by_region = nhl_df.groupby("Metropolitan area").agg(
        {"W": "sum", "L": "sum", "W/L": np.mean}
    )  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region["winloss_new"] = win_loss_by_region["W"] / (
        win_loss_by_region["W"] + win_loss_by_region["L"]
    )
    win_loss_by_region = win_loss_by_region["winloss_new"]

    # print('length of population_by_region', len(population_by_region))
    # print(population_by_region)
    # print('length of win_loss_by_region', len(win_loss_by_region))
    # print(win_loss_by_region)

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q1: Your lists must be the same length"
    assert (
        len(population_by_region) == 28
    ), "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


print(nhl_correlation())

0.01230899645574425


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import re
from io import StringIO

import numpy as np
import pandas as pd
import scipy.stats as stats


def clear_special(word):
    word = re.sub(r"\[.*?\]", "", word)  # remove all superscript characters
    word = re.sub(r"[^A-Za-z\s,]*", "", word)
    return word


def nba_correlation():
    nba_df = pd.read_csv("assets/nba.csv")
    # Open the HTML file and read it as a text file
    with open("assets/wikipedia_data.html", "r", encoding="utf-8") as f:
        html_file = f.read()
    html_file = re.sub(
        r"<br>", ",", html_file
    )  # to get multiple teams in the same city in the same cell
    # print(html_file)
    html_io = StringIO(html_file)
    cities = pd.read_html(html_io)[1]

    cities = cities.iloc[
        :-1, [0, 3, 5, 6, 7, 8]
    ]  # cities.iloc[:-1] removes the last row
    mapping_dic = {
        "New York City Nets": "Brooklyn Nets",
        "Dallas–Fort Worth Mavericks": "Dallas Mavericks",
        "San Francisco Bay Area Warriors": "Golden State Warriors",
        "Indianapolis Pacers": "Indiana Pacers",
        "Miami–Fort Lauderdale Heat": "Miami Heat",
        "Minneapolis–Saint Paul Timberwolves": "Minnesota Timberwolves",
        "New York City Knicks": "New York Knicks",
        "Salt Lake City Jazz": "Utah Jazz",
        "Washington, D.C. Wizards": "Washington Wizards",
    }

    cities.rename(columns={"Population ,(2016 est.)[8]": "population"}, inplace=True)
    cities["NBA"] = cities["NBA"].apply(clear_special)
    cities = cities.assign(NBA=cities["NBA"].str.split(",")).explode("NBA")
    cities["NBA"] = cities["Metropolitan area"] + " " + cities["NBA"]

    cities = cities[
        ["Metropolitan area", "population", "NBA"]
    ]  # include only the columns we need
    cities = cities.replace({"NBA": mapping_dic})
    cities.sort_values(by="NBA", inplace=True)

    nba_df["team"] = nba_df["team"].apply(
        clear_special
    )  # remove * and then strip spaces
    nba_df = nba_df[(nba_df["W"].str.isnumeric()) & (nba_df["L"].str.isnumeric())]
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df = nba_df[["team", "W", "L"]]  # include only the columns we need
    nba_df["winlossratio"] = nba_df["W"].astype(float) / (
        nba_df["W"].astype(float) + nba_df["L"].astype(float)
    )
    nba_df.sort_values(by="team", inplace=True)
    # print('Cities Data Frame and its length=',len(cities))
    # print(cities)
    # print('nba_df Data Frame and its length=',len(nba_df))
    # print(nba_df)

    cities["NBA"] = cities["NBA"].str.strip()
    nba_df["team"] = nba_df["team"].str.strip()
    merged_df = pd.merge(cities, nba_df, how="inner", left_on="NBA", right_on="team")
    merged_df.sort_values(by="NBA", inplace=True)
    # print('merged_df Data Frame and its length=',len(merged_df))
    # print(merged_df)

    summary_df = merged_df.groupby("Metropolitan area").agg(
        {"population": "first", "winlossratio": "mean"}
    )
    summary_df["population"] = pd.to_numeric(summary_df["population"], errors="coerce")

    population_by_region = summary_df[
        "population"
    ]  # pass in metropolitan area population from cities
    win_loss_by_region = summary_df[
        "winlossratio"
    ]  # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    # print('Summary Data Frame and its length=',len(summary_df))
    # print(summary_df)

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q1: Your lists must be the same length"
    assert (
        len(population_by_region) == 28
    ), "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import re
from io import StringIO

import numpy as np
import pandas as pd
import scipy.stats as stats


def clear_special(word):
    word = re.sub(r"\[.*?\]", "", word)  # remove all superscript characters
    word = re.sub(r"[^A-Za-z\s,]*", "", word)
    return word


def mlb_correlation():
    mlb_df = pd.read_csv("assets/mlb.csv")
    # Open the HTML file and read it as a text file
    with open("assets/wikipedia_data.html", "r", encoding="utf-8") as f:
        html_file = f.read()
    html_file = re.sub(
        r"<br>", ",", html_file
    )  # to get multiple teams in the same city in the same cell
    # print(html_file)
    html_io = StringIO(html_file)
    cities = pd.read_html(html_io)[1]

    cities = cities.iloc[
        :-1, [0, 3, 5, 6, 7, 8]
    ]  # cities.iloc[:-1] removes the last row
    mapping_dic = {
        "Phoenix Diamondbacks": "Arizona Diamondbacks",
        "Denver Rockies": "Colorado Rockies",
        "Miami–Fort Lauderdale Marlins": "Miami Marlins",
        "Minneapolis–Saint Paul Twins": "Minnesota Twins",
        "New York City Mets": "New York Mets",
        "New York City Yankees": "New York Yankees",
        "San Francisco Bay Area Athletics": "Oakland Athletics",
        "San Francisco Bay Area Giants": "San Francisco Giants",
        "St. Louis Cardinals": "St Louis Cardinals",
        "Tampa Bay Area Rays": "Tampa Bay Rays",
        "Dallas–Fort Worth Rangers": "Texas Rangers",
        "Washington, D.C. Nationals": "Washington Nationals",
    }

    cities.rename(columns={"Population ,(2016 est.)[8]": "population"}, inplace=True)
    cities["MLB"] = cities["MLB"].apply(clear_special)
    cities = cities.assign(MLB=cities["MLB"].str.split(",")).explode("MLB")
    cities["MLB"] = cities["Metropolitan area"] + " " + cities["MLB"]

    cities = cities[
        ["Metropolitan area", "population", "MLB"]
    ]  # include only the columns we need
    cities = cities.replace({"MLB": mapping_dic})
    cities.sort_values(by="MLB", inplace=True)

    mlb_df["team"] = mlb_df["team"].apply(
        clear_special
    )  # remove * and then strip spaces
    # mlb_df = mlb_df[(mlb_df['W'].str.isnumeric()) & (mlb_df['L'].str.isnumeric())]
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df = mlb_df[["team", "W", "L"]]  # include only the columns we need
    mlb_df["winlossratio"] = mlb_df["W"].astype(float) / (
        mlb_df["W"].astype(float) + mlb_df["L"].astype(float)
    )
    mlb_df.sort_values(by="team", inplace=True)
    # print('Cities Data Frame and its length=',len(cities))
    # print(cities)
    # print('mlb_df Data Frame and its length=',len(mlb_df))
    # print(mlb_df)

    cities["MLB"] = cities["MLB"].str.strip()
    mlb_df["team"] = mlb_df["team"].str.strip()
    merged_df = pd.merge(cities, mlb_df, how="inner", left_on="MLB", right_on="team")
    merged_df.sort_values(by="MLB", inplace=True)
    # print('merged_df Data Frame and its length=',len(merged_df))
    # print(merged_df)

    summary_df = merged_df.groupby("Metropolitan area").agg(
        {"population": "first", "winlossratio": "mean"}
    )
    summary_df["population"] = pd.to_numeric(summary_df["population"], errors="coerce")

    population_by_region = summary_df[
        "population"
    ]  # pass in metropolitan area population from cities
    win_loss_by_region = summary_df[
        "winlossratio"
    ]  # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    # print('Summary Data Frame and its length=',len(summary_df))
    # print(summary_df)

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q3: Your lists must be the same length"
    assert (
        len(population_by_region) == 26
    ), "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import re
from io import StringIO

import numpy as np
import pandas as pd
import scipy.stats as stats


def clear_special(word):
    word = re.sub(r"\[.*?\]", "", word)  # remove all superscript characters
    word = re.sub(r"[^A-Za-z\s,]*", "", word)
    return word


def nfl_correlation():
    nfl_df = pd.read_csv("assets/nfl.csv")
    # Open the HTML file and read it as a text file
    with open("assets/wikipedia_data.html", "r", encoding="utf-8") as f:
        html_file = f.read()
    html_file = re.sub(
        r"<br>", ",", html_file
    )  # to get multiple teams in the same city in the same cell
    # print(html_file)
    html_io = StringIO(html_file)
    cities = pd.read_html(html_io)[1]

    cities = cities.iloc[
        :-1, [0, 3, 5, 6, 7, 8]
    ]  # cities.iloc[:-1] removes the last row
    mapping_dic = {
        "Phoenix Cardinals": "Arizona Cardinals",
        "Charlotte Panthers": "Carolina Panthers",
        "Dallas–Fort Worth Cowboys": "Dallas Cowboys",
        "Miami–Fort Lauderdale Dolphins": "Miami Dolphins",
        "Minneapolis–Saint Paul Vikings": "Minnesota Vikings",
        "New York City Giants": "New York Giants",
        "Boston Patriots": "New England Patriots",
        "New York City Jets": "New York Jets",
        "San Francisco Bay Area Raiders": "Oakland Raiders",
        "San Francisco Bay Area ers": "San Francisco ers",
        "Tampa Bay Area Buccaneers": "Tampa Bay Buccaneers",
        "Nashville Titans": "Tennessee Titans",
        "Washington, D.C. Redskins": "Washington Redskins",
    }

    cities.rename(columns={"Population ,(2016 est.)[8]": "population"}, inplace=True)
    cities["NFL"] = cities["NFL"].apply(clear_special)
    cities = cities.assign(NFL=cities["NFL"].str.split(",")).explode("NFL")
    cities["NFL"] = cities["Metropolitan area"] + " " + cities["NFL"]

    cities = cities[
        ["Metropolitan area", "population", "NFL"]
    ]  # include only the columns we need
    cities = cities.replace({"NFL": mapping_dic})
    cities.sort_values(by="NFL", inplace=True)

    nfl_df["team"] = nfl_df["team"].apply(
        clear_special
    )  # remove * and then strip spaces
    nfl_df = nfl_df[(nfl_df["W"].str.isnumeric()) & (nfl_df["L"].str.isnumeric())]
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df = nfl_df[["team", "W", "L"]]  # include only the columns we need
    nfl_df["winlossratio"] = nfl_df["W"].astype(float) / (
        nfl_df["W"].astype(float) + nfl_df["L"].astype(float)
    )
    nfl_df.sort_values(by="team", inplace=True)
    # print('Cities Data Frame and its length=',len(cities))
    # print(cities)
    # print('nfl_df Data Frame and its length=',len(nfl_df))
    # print(nfl_df)

    cities["NFL"] = cities["NFL"].str.strip()
    nfl_df["team"] = nfl_df["team"].str.strip()
    merged_df = pd.merge(cities, nfl_df, how="inner", left_on="NFL", right_on="team")
    merged_df.sort_values(by="NFL", inplace=True)
    # print('merged_df Data Frame and its length=',len(merged_df))
    # print(merged_df)

    summary_df = merged_df.groupby("Metropolitan area").agg(
        {"population": "first", "winlossratio": "mean"}
    )
    summary_df["population"] = pd.to_numeric(summary_df["population"], errors="coerce")

    population_by_region = summary_df[
        "population"
    ]  # pass in metropolitan area population from cities
    win_loss_by_region = summary_df[
        "winlossratio"
    ]  # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    # print('Summary Data Frame and its length=',len(summary_df))
    # print(summary_df)

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q4: Your lists must be the same length"
    assert (
        len(population_by_region) == 29
    ), "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nfl_correlation()

0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [6]:
import re
from io import StringIO

import numpy as np
import pandas as pd
import scipy.stats as stats


def clear_special(word):
    word = re.sub(r"\[.*?\]", "", word)  # remove all superscript characters
    word = re.sub(r"[^A-Za-z\s,]*", "", word)
    return word


def perfrom_sports(sport_df):
    sport_df["team"] = sport_df["team"].apply(
        clear_special
    )  # remove * and then strip spaces
    sport_df = sport_df[sport_df["year"] == 2018]
    try:
        sport_df = sport_df[
            (sport_df["W"].str.isnumeric()) & (sport_df["L"].str.isnumeric())
        ]
    except Exception as e:
        print("Exception in sport_df", e)
    sport_df = sport_df[["team", "W", "L"]]  # include only the columns we need
    sport_df["winlossratio"] = sport_df["W"].astype(float) / (
        sport_df["W"].astype(float) + sport_df["L"].astype(float)
    )
    sport_df.sort_values(by="team", inplace=True)
    return sport_df


def sports_team_performance():
    mlb_df = pd.read_csv("assets/mlb.csv")
    nhl_df = pd.read_csv("assets/nhl.csv")
    nba_df = pd.read_csv("assets/nba.csv")
    nfl_df = pd.read_csv("assets/nfl.csv")

    sports = ["NFL", "NBA", "NHL", "MLB"]
    sports_dfs = [nfl_df, nba_df, nhl_df, mlb_df]
    merged_nfl_df, merged_nba_df, merged_nhl_df, merged_mlb_df = (
        pd.DataFrame(),
        pd.DataFrame(),
        pd.DataFrame(),
        pd.DataFrame(),
    )
    merged_dfs = [merged_nfl_df, merged_nba_df, merged_nhl_df, merged_mlb_df]
    # summary_nfl_df, summary_nba_df, summary_nhl_df, summary_mlb_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    # summary_dfs = [summary_nfl_df, summary_nba_df, summary_nhl_df, summary_mlb_df]

    # Open the HTML file and read it as a text file
    with open("assets/wikipedia_data.html", "r", encoding="utf-8") as f:
        html_file = f.read()
    html_file = re.sub(
        r"<br>", ",", html_file
    )  # to get multiple teams in the same city in the same cell
    # print(html_file)
    html_io = StringIO(html_file)
    cities = pd.read_html(html_io)[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    mapping = {
        "NFL": {
            "Phoenix Cardinals": "Arizona Cardinals",
            "Charlotte Panthers": "Carolina Panthers",
            "Dallas–Fort Worth Cowboys": "Dallas Cowboys",
            "Miami–Fort Lauderdale Dolphins": "Miami Dolphins",
            "Minneapolis–Saint Paul Vikings": "Minnesota Vikings",
            "New York City Giants": "New York Giants",
            "Boston Patriots": "New England Patriots",
            "New York City Jets": "New York Jets",
            "San Francisco Bay Area Raiders": "Oakland Raiders",
            "San Francisco Bay Area ers": "San Francisco ers",
            "Tampa Bay Area Buccaneers": "Tampa Bay Buccaneers",
            "Nashville Titans": "Tennessee Titans",
            "Washington, D.C. Redskins": "Washington Redskins",
        },
        "NHL": {
            "Tampa Bay Area Lightning": "Tampa Bay Lightning",
            "Los Angeles Ducks": "Anaheim Ducks",
            "Phoenix Coyotes": "Arizona Coyotes",
            "Raleigh Hurricanes": "Carolina Hurricanes",
            "Denver Avalanche": "Colorado Avalanche",
            "Miami–Fort Lauderdale Panthers": "Florida Panthers",
            "Minneapolis–Saint Paul Wild": "Minnesota Wild",
            "New York City Devils": "New Jersey Devils",
            "New York City Islanders": "New York Islanders",
            "New York City Rangers": "New York Rangers",
            "San Francisco Bay Area Sharks": "San Jose Sharks",
            "Las Vegas Golden Knights": "Vegas Golden Knights",
            "Washington, D.C. Capitals": "Washington Capitals",
            "Dallas–Fort Worth Stars": "Dallas Stars",
        },
        "NBA": {
            "New York City Nets": "Brooklyn Nets",
            "Dallas–Fort Worth Mavericks": "Dallas Mavericks",
            "San Francisco Bay Area Warriors": "Golden State Warriors",
            "Indianapolis Pacers": "Indiana Pacers",
            "Miami–Fort Lauderdale Heat": "Miami Heat",
            "Minneapolis–Saint Paul Timberwolves": "Minnesota Timberwolves",
            "New York City Knicks": "New York Knicks",
            "Salt Lake City Jazz": "Utah Jazz",
            "Washington, D.C. Wizards": "Washington Wizards",
        },
        "MLB": {
            "Phoenix Diamondbacks": "Arizona Diamondbacks",
            "Denver Rockies": "Colorado Rockies",
            "Miami–Fort Lauderdale Marlins": "Miami Marlins",
            "Minneapolis–Saint Paul Twins": "Minnesota Twins",
            "New York City Mets": "New York Mets",
            "New York City Yankees": "New York Yankees",
            "San Francisco Bay Area Athletics": "Oakland Athletics",
            "San Francisco Bay Area Giants": "San Francisco Giants",
            "St. Louis Cardinals": "St Louis Cardinals",
            "Tampa Bay Area Rays": "Tampa Bay Rays",
            "Dallas–Fort Worth Rangers": "Texas Rangers",
            "Washington, D.C. Nationals": "Washington Nationals",
        },
    }

    cities.rename(columns={"Population ,(2016 est.)[8]": "population"}, inplace=True)
    for key, value in mapping.items():
        cities[key] = cities[key].apply(clear_special)
        cities = cities.assign(**{key: cities[key].str.split(",")}).explode(key)
        cities[key] = cities["Metropolitan area"] + " " + cities[key]
        cities = cities.replace({key: value})
        cities[key] = cities[key].str.strip()

    cities = cities[
        ["Metropolitan area", "population", "NFL", "NBA", "NHL", "MLB"]
    ]  # include only the columns we need
    cities.sort_values(by="Metropolitan area", inplace=True)
    print("Cities Data Frame and its length=", len(cities))
    print(cities)

    i = 0
    while i < 4:
        sports_dfs[i] = perfrom_sports(sports_dfs[i])  # remove * and then strip spaces
        i += 1

    for sport in sports:
        cities[sport] = cities[sport].str.strip()
    for sport_df in sports_dfs:
        sport_df["team"] = sport_df["team"].str.strip()

    i = 0
    while i < 4:
        merged_dfs[i] = pd.merge(
            cities, sports_dfs[i], how="inner", left_on=sports[i], right_on="team"
        )
        merged_dfs[i].sort_values(by="Metropolitan area", inplace=True)
        merged_dfs[i] = (
            merged_dfs[i]
            .groupby(["Metropolitan area"])
            .agg(
                {
                    "Metropolitan area": "first",
                    "population": "first",
                    "winlossratio": "mean",
                    sports[i]: "nunique",
                }
            )
        )
        # merged_dfs[i].reset_index(inplace=True)

        merged_dfs[i].to_csv("assets/output_temp" + str(i) + ".csv", index=False)
        print(
            f"merged_df loop {sports[i]} Data Frame and its length=", len(merged_dfs[i])
        )
        print(merged_dfs[i])
        i += 1

    columns = ["Metropolitan area", "population", "NFL", "NBA", "NHL", "MLB"]
    # summary_df = merged_nfl_df.merge(merged_nba_df, on='Metropolitan area', how='outer')
    # print('Summary Data Frame and its length=',len(summary_df))
    # print(summary_df)
    # merged_df = merged_df.merge(df3, on='common_column', how='outer')
    # merged_df = merged_df.merge(df4, on='common_column', how='outer')
    # while i < 4:
    #     summary_dfs[i] = merged_dfs[i].groupby('Metropolitan area').agg({'Metropolitan area':'first','population': 'first', 'winlossratio': 'mean'})
    #     merged_dfs[i].sort_values(by='Metropolitan area', inplace=True)
    #     merged_dfs[i].to_csv('assets/output_temp'+str(i)+'.csv', index=False)
    #     print(f'merged_df loop {sports[i]} Data Frame and its length=',len(merged_dfs[i]))
    #     print(merged_dfs[i])
    #     i+=1
    # summary_df = merged_df.groupby('Metropolitan area').agg({'population': 'first', 'winlossratio': 'mean'})
    # for sport in ['NBA', 'NHL', 'MLB','NFL']:
    #     summary_df[sport+'_team_count'] = merged_df.groupby('Metropolitan area')[sport].nunique()
    # summary_df['population'] = pd.to_numeric(summary_df['population'], errors='coerce')

    # population_by_region = summary_df["population"] # pass in metropolitan area population from cities
    # win_loss_by_region = summary_df['winlossratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    # print('Summary Data Frame and its length=',len(summary_df))
    # print(summary_df)

    # from scipy.stats import ttest_rel
    # # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # # df.loc["NFL","NFL"] should return np.nan
    # sports = ['NFL', 'NBA', 'NHL', 'MLB']
    # p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    # print('p_values Data Frame and its length=',len(p_values))
    # print(p_values)

    # assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    # assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    # return p_values


sports_team_performance()

Cities Data Frame and its length= 93
   Metropolitan area population                   NFL                 NBA  \
15           Atlanta    5789700       Atlanta Falcons       Atlanta Hawks   
23         Baltimore    2798886      Baltimore Ravens           Baltimore   
7             Boston    4794447  New England Patriots      Boston Celtics   
29           Buffalo    1132804         Buffalo Bills             Buffalo   
35           Calgary    1392609               Calgary             Calgary   
..               ...        ...                   ...                 ...   
16    Tampa Bay Area    3032171  Tampa Bay Buccaneers      Tampa Bay Area   
13           Toronto    5928040               Toronto     Toronto Raptors   
31         Vancouver    2463431             Vancouver           Vancouver   
5   Washington, D.C.    6131977   Washington Redskins  Washington Wizards   
39          Winnipeg     778489              Winnipeg            Winnipeg   

                    NHL               